# Music Genre Classification

* Install Miniconda from their website
* Put Conda on PATH during the install
* Update conda - Terminal Command: conda update conda
* Clone the Github repository (send Tim or I your github username if you haven’t yet).
* Load environment.yaml into conda environment (this will take a while to download) - Terminal Command: conda env create -n mus2vid -f environment.yaml

## Download CSV file with Classical Dataset and Parse

In [21]:
import pandas as pd

classical_data = pd.read_csv('classical_dataset.csv')
# print (classical_data)

midis = list(classical_data['midi_filename'])
years = list(classical_data['Average'])
for i in range(0, len(midis)):
    midis[i] = "maestro-v3.0.0/" + midis[i]
print(midis)
print(years)
print(len(midis))

['maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi', 'maestro-v3.0.0/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi', 'maestro-v3.0.0/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi', 'maestro-v3.0.0/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi', 'maestro-v3.0.0/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi', 'maestro-v3.0.0/2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi', 'maestro-v3.0.0/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi', 'maestro-v3.0.0/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi', 'maestro-v3.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi', 'maestro-v3.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.mid

## Process wav files and convert to MIDI objects


In [22]:
# Zelin's code will create a list of .wav filepaths and years of composition
# temporarily name these wavs and years
import pickle
import pretty_midi as pm
import pandas as pd
import numpy as np
import tensorflow as tf
# from basic_pitch.inference import predict as bp_predict
# from basic_pitch import ICASSP_2022_MODEL_PATH
# BASIC_PITCH_MODEL = tf.saved_model.load(str(ICASSP_2022_MODEL_PATH))

# Parameters
STD_ONSET = 0.3
STD_FRAME = 0.2
STD_MIN_NOTE_LEN = 50
STD_MIN_FREQ = None
STD_MAX_FREQ = 3000

# def predict(filestr):
#     """
#     Makes a Basic Pitch prediction with the global parameters above given an input audio file.
    
#     Parameters:
#         filestr (str): The path to the input audio file.
        
#     Returns:
#         PrettyMIDI object containing predicted MIDI notes.
#     """
#     # Run prediction
#     model_output, midi_data, note_events = bp_predict(
#         filestr,
#         BASIC_PITCH_MODEL,
#         STD_ONSET,
#         STD_FRAME,
#         STD_MIN_NOTE_LEN,
#         STD_MIN_FREQ,
#         STD_MAX_FREQ
#     ) # midi_data is the PrettyMIDI object corresponding to the prediction
#     return midi_data

def get_matched_midi(wavs, years):
    """
    Creates pretty-midi objects from input .wav files
    Matches each MIDI object with a composition year based on the years list

    Parameters:
        wavs (list): list of .wav filepaths.
        years (list): list of (approximate) composition years for each .wav file

    Returns:
        pandas.DataFrame: A DataFrame containing the pretty_midi obj and the corresponding composition year
    """
    prettys = [] # list of pretty_midi objects
    for filepath in wavs:
        midi_obj = pm.PrettyMIDI(filepath)
        prettys.append(midi_obj)

    df = pd.DataFrame({'midi_obj': prettys, 'year': years})

    return df

matched_midi_df = get_matched_midi(midis, years)

# Optionally save the matched_midi_df DataFrame as a pickle file
with open("matched_midi.pkl", "wb") as f:
    pickle.dump(matched_midi_df, f)

# Print the first few rows
print(matched_midi_df.head())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
import pickle

# If saved, load matched_midi_df from the saved pickle file
with open('matched_midi.pkl', 'rb') as f:
    matched_midi_df = pickle.load(f)

## Extract Midi Files Features

In [24]:
import pickle
import numpy as np
import pretty_midi
from sklearn.utils import resample

def normalize_features(features):
    """
    Normalizes the features to the range [-1, 1].

    Parameters:
        features (list of float): The array of features.

    Returns:
        list of float: Normalized features.
    """
    # Normalize each feature based on its specific range
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    melody_complexity = (features[5] - 0) / 10
    melody_range = (features[6] - 0) / 80

    # Normalize pitch class histogram
    pitch_class_hist = [((f - 0) / 100) for f in features[7:-1]]

    # Return the normalized feature vector
    return [tempo, resolution, time_sig_1, time_sig_2, melody_complexity, melody_range] + pitch_class_hist

def get_features(midi_obj):
    """
    Extracts specific features from a PrettyMIDI object given its path using the pretty_midi library.
    Handle any potential errors with MIDI files appropriately.

    Parameters:
        midi_obj: the PrettyMIDI object

    Returns:
        list of float: The extracted features.
    """
    
    # tempo: the estimated tempo of the audio file
    tempo = midi_obj.estimate_tempo()

    # num_sig_changes: the number of time signature changes in the audio file
    num_sig_changes = len(midi_obj.time_signature_changes)

    # resolution: the time resolution of the audio file (in ticks per beat)
    resolution = midi_obj.resolution


    # Extract time signature information
    ts_changes = midi_obj.time_signature_changes
    ts_1, ts_2 = 4, 4
    if len(ts_changes) > 0:
        ts_1 = ts_changes[0].numerator
        ts_2 = ts_changes[0].denominator
    
    # Extract melody-related features
    # melody: a pitch class histogram of the audio file
    melody = midi_obj.get_pitch_class_histogram()
    # melody_complexity: the number of unique pitch classes in the melody
    melody_complexity = np.sum(melody > 0)
    # melody_range: the range of pitch classes in the melody
    melody_range = np.max(melody) - np.min(melody)
    # OPTIONAL feature melody_contour: the temporal evolution of pitch content in the audio file
    # melody_contour = librosa.feature.tempogram(y=file.fluidsynth(fs=16000), sr=16000, hop_length=512)
    # melody_contour = np.mean(melody_contour, axis=0)
    # chroma: a chroma representation of the audio file
    chroma = midi_obj.get_chroma()
    # pitch_class_hist: the sum of the chroma matrix along the pitch axis
    pitch_class_hist = np.sum(chroma, axis=1)

    return normalize_features([tempo, num_sig_changes, resolution, ts_1,
                    ts_2, melody_complexity, melody_range] + list(pitch_class_hist)) # + list(melody_contour))

def extract_midi_features(midi_df):
    """
    Extracts features and labels from MIDI objects listed in the DataFrame and concatenates the
    features with their labels into a matrix.

    Since the dataset is inherently unbalanced in terms of genre distribution, oversampling and 
    undersampling can be used to achieve a more balanced representation of features for each genre.

    Parameters:
        path_df (pandas.DataFrame): A DataFrame with pretty-midi objects and their composition years.

    Returns:
        numpy.ndarray: A matrix of features along with labels.
    """
    all_features = []  # List to store all extracted features
    iters = 0
    for index, row in midi_df.iterrows():
        print(iters)
        iters += 1
        obj_features = get_features(row['midi_obj'])
        obj_features.append(row['year'])
        all_features.append(obj_features)
    # Return the numpy array of all extracted features along with corresponding years
    return np.array(all_features)

# Call the extract_midi_features function with the appropriate path DataFrame to extract the MIDI 
# file features and obtain the feature-label matrix
labeled_features = extract_midi_features(matched_midi_df)
# Print the labeled features
print(labeled_features)

# Optionally store the feature-label matrix as a pickle file for further use
with open('labeled_features_over.pkl', 'wb') as f:
    pickle.dump(labeled_features, f)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
import pickle

# If saved, load matched_midi_df from the saved pickle file
with open('labeled_features.pkl', 'rb') as f:
    labeled_features = pickle.load(f)

print(len(labeled_features))

## Partition Dataset into Training, Validation, and Testing

In [6]:
from keras.utils import to_categorical
import numpy as np

# Shuffle the features
labeled_features = np.random.permutation(labeled_features)

# Partition the Dataset into 3 Sets: Training, Validation, and Test
num = len(labeled_features)
# Calculate the number of samples for training data (60% of the dataset)
num_training = int(num * 0.6)
# Calculate the number of samples for validation data (20% of the dataset)
num_validation = int(num * 0.8)

# Extract the training data (60% of the labeled features)
training_data = labeled_features[:num_training]
# Extract the validation data (20% of the labeled features)
validation_data = labeled_features[num_training:num_validation]
# Extract the test data (remaining 20% of the labeled features)
test_data = labeled_features[num_validation:]


# Separate the features from the labels
num_cols = training_data.shape[1] - 1
# Extract features from the training data
training_features = training_data[:, :num_cols]
# Extract features from the validation data
validation_features = validation_data[:, :num_cols]
# Extract features from the test data
test_features = test_data[:, :num_cols]

# Extract years from the training data
training_labels = training_data[:, num_cols]
# Extract years from the validation data
validation_labels = validation_data[:, num_cols]
# Extract years from the test data
test_labels = test_data[:, num_cols]

print(test_features[:10])  # Print the first 10 rows of test features
print(test_labels[:10])  # Print the first 10 test labels
# print(to_categorical((test_labels)[:10]))  # Print the one-hot encoding of the first 10 test labels


[[ 9.63142102e-02 -3.50000000e-01  1.25000000e-01  1.25000000e-01
   1.20000000e+00  3.28393136e-03  2.90862843e+04  3.84259595e+02
   7.67716633e+03  5.93171646e+03  5.13025432e+03  1.35359018e+04
   8.74087583e+02  2.56013226e+04  1.45344695e+02  3.38658483e+03
   7.71258439e+03]
 [ 2.50000000e-01 -3.50000000e-01  1.25000000e-01  1.25000000e-01
   8.00000000e-01  2.39785319e-03  2.35381800e+04  0.00000000e+00
   3.40029800e+04  0.00000000e+00  8.49122000e+03  2.05917800e+04
   0.00000000e+00  2.37896400e+04  0.00000000e+00  1.65608000e+04
   2.03200000e+04]
 [ 9.21938089e-02 -5.30000000e-01  1.25000000e-01  1.25000000e-01
   1.00000000e+00  5.29978587e-03  4.31800000e+01  1.71450000e+03
   4.65201000e+03  2.66408545e+01  1.15529311e+04  1.93380444e+01
   2.88290000e+02  0.00000000e+00  2.24155000e+03  1.46670957e+04
   1.45424116e+03]
 [ 1.20149481e-01 -4.10000000e-01  1.25000000e-01  1.25000000e-01
   1.20000000e+00  3.81565907e-03  1.60582000e+03  4.25184000e+03
   4.15736000e+03  

In [2]:
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder  
from keras.utils import to_categorical
import pickle


# Define the model architecture
model = keras.Sequential([
    # First hidden layer with 256 units and ReLU activation
    keras.layers.Dense(256, input_shape=(training_features.shape[1],), activation='relu'), # try relu and sigmoid
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Second hidden layer with 128 units and ReLU activation
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Third hidden layer with 64 units and ReLU activation
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Output layer
    keras.layers.Dense(1, activation='relu')
])

# Print a summary of the model architecture
print(model.summary())


"""
optimizer="adam": The optimizer algorithm to use during training. 
Adam optimizer is chosen, which is a popular optimization algorithm known for its efficiency.

loss='categorical_crossentropy': The loss function used to measure the discrepancy between the 
predicted output and the true output labels. Categorical cross-entropy is suitable for
multi-class classification tasks.

metrics=['accuracy']: The metric(s) to be evaluated during training and testing. 
Accuracy is a commonly used metric to assess the model's performance.
"""

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Encode the training and validation labels using one-hot encoding
# train_labels_encoded = to_categorical(training_labels)
# val_labels_encoded = to_categorical(validation_labels)


"""
training_features, train_labels_encoded: Input features and corresponding labels for model training.

validation_features, val_labels_encoded: Validation set used to monitor the model's performance 
                                         during training.

batch_size=32: Number of samples per gradient update. Training data is divided into batches, 
               and the model's weights are updated after each batch.

epochs=50: Number of times the model will iterate over the entire training dataset.

callbacks: EarlyStopping to stop training if the validation loss does not improve for a certain 
           number of epochs, and ModelCheckpoint to save the best model based on validation loss.
"""
history = model.fit(training_features, train_labels_encoded, 
                    validation_data=(validation_features, val_labels_encoded),
                    batch_size=32, epochs=50, verbose=2,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
                               keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)])

# Save the entire model to an h5 file
model.save("my_model.h5")

# Optionally save the entire model as a pickle file
with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)


# Optionally load the pickled model from file
with open('my_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Use the loaded model for prediction
preds = model.predict(test_features[:1])
print(test_features[:1].shape)

print("predictions: ")
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print(preds)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_features, to_categorical(test_labels))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

KeyboardInterrupt: 

Classify A New MIDI File Using NN

In [5]:
import pickle

with open('matched_midi.pkl', 'rb') as f:
    matched_midi_df = pickle.load(f)

with open('labeled_features.pkl', 'rb') as f:
    labeled_features = pickle.load(f)

with open('my_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [46]:
import numpy as np
import pretty_midi

def normalize_features(features):
    """
    Normalizes the features to the range [-1, 1].

    Parameters:
        features (list of float): The array of features.

    Returns:
        list of float: Normalized features.
    """
    # Normalize each feature based on its specific range
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    melody_complexity = (features[5] - 0) / 10
    melody_range = (features[6] - 0) / 80

    # Normalize pitch class histogram
    pitch_class_hist = [((f - 0) / 100) for f in features[7:-1]]

    # Return the normalized feature vector
    return [tempo, resolution, time_sig_1, time_sig_2, melody_complexity, melody_range] + pitch_class_hist

def get_features(path):
    """
    Extracts specific features from a MIDI file given its path using the pretty_midi library.
    Handle any potential errors with MIDI files appropriately.

    Parameters:
        path (str): The path to the MIDI file.

    Returns:
        list of float: The extracted features.
    """
    # Creates a PrettyMIDI object by loading the MIDI file specified by the given path.
    file = pretty_midi.PrettyMIDI(path)
    
    # tempo: the estimated tempo of the audio file
    tempo = file.estimate_tempo()

    # num_sig_changes: the number of time signature changes in the audio file
    num_sig_changes = len(file.time_signature_changes)

    # resolution: the time resolution of the audio file (in ticks per beat)
    resolution = file.resolution


    # Extract time signature information
    ts_changes = file.time_signature_changes
    ts_1, ts_2 = 4, 4
    if len(ts_changes) > 0:
        ts_1 = ts_changes[0].numerator
        ts_2 = ts_changes[0].denominator
    
    # Extract melody-related features
    # melody: a pitch class histogram of the audio file
    melody = file.get_pitch_class_histogram()
    # melody_complexity: the number of unique pitch classes in the melody
    melody_complexity = np.sum(melody > 0)
    # melody_range: the range of pitch classes in the melody
    melody_range = np.max(melody) - np.min(melody)
    # OPTIONAL feature melody_contour: the temporal evolution of pitch content in the audio file
    # melody_contour = librosa.feature.tempogram(y=file.fluidsynth(fs=16000), sr=16000, hop_length=512)
    # melody_contour = np.mean(melody_contour, axis=0)
    # chroma: a chroma representation of the audio file
    chroma = file.get_chroma()
    # pitch_class_hist: the sum of the chroma matrix along the pitch axis
    pitch_class_hist = np.sum(chroma, axis=1)

    return normalize_features([tempo, num_sig_changes, resolution, ts_1,
                    ts_2, melody_complexity, melody_range] + list(pitch_class_hist)) # + list(melody_contour))
    
midi_path = "test.mid"
midi_features = np.asarray(get_features(midi_path))
midi_features = np.expand_dims(midi_features, axis = 0)

prediction = model.predict(midi_features)

print(prediction)

genre = np.argmax(prediction)
print(label_list[genre])


1/1 [==============================] - 0s 24ms/step
[[0.008 0.581 0.004 0.154 0.037 0.075 0.031 0.015 0.048 0.01  0.003 0.003
  0.031]]
Country
